In [ ]:
# set the seed value for the notebook so results are reproducible
from numpy.random import seed
seed(42)

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# read in the data
movies = pd.read_csv('moviesClean.csv')


movies.head()

In [ ]:
# drop text-based columns for the model
movies.drop(columns=['original_title', 'genre', 'country', 'language',
                     'director', 'writer', 'production_company', 'actors'], inplace=True)
movies.shape

In [ ]:
# from sklearn.datasets import make_classification

X = movies.drop('success', axis = 1)
y = movies['success']

y = y.values.reshape(-1, 1)

print(X.shape, y.shape)

In [ ]:
# create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#   Data Pre-Processing

In [ ]:
# scale the data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

In [ ]:
# scale both training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# One-hot encoding
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

# Create the Model

In [ ]:
# create a sequential model
from tensorflow.keras.models import Sequential

model = Sequential()

In [ ]:
# add the first layer. The number of inputs must be equal to the
# number of columns
from tensorflow.keras.layers import Dense
number_inputs = 10484
number_hidden_nodes = 10
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [ ]:
# add the final layer. number_classes is the number of labels to predict.
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

# Model Summary

In [ ]:
model.summary()

# Compile the Model

In [ ]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the Model

In [ ]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

# Quantify/Test the Model

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Make Predictions with New Data?

In [ ]:
# need sample data. Maybe we can use movies from 2020?

# print(f"Predicted class: {model.predict_classes(new_data)}")

# Save the Model

In [ ]:
# model.save('classification_neural_network.h5')

In [ ]:
# # Load the model
# from tensorflow.keras.models import load_model
# voice_model = load_model("classification_neural_network.h5")